Set up

In [1]:
#install lib
!pip install google-api-python-client pandas python-dotenv tqdm isodate pyarrow yt-dlp gspread

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 24.3 MB/s eta 0:00:00


In [2]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import & Config


In [16]:
import os, json, glob
import pandas as pd
from yt_dlp import YoutubeDL
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
import gspread
from dotenv import load_dotenv

env_path = "/content/drive/MyDrive/collect_data_yt_toxic_cmt/.env"
load_dotenv(env_path)

api_key = os.getenv('YOUTUBE_API_KEY')
Sheet_url = "https://docs.google.com/spreadsheets/d/1LIrMrc6jOaF87AMaWnABeFxUAWY-WrApYVo1D9D9mJ8/edit?gid=0#gid=0"
Sheet_id = Sheet_url.split("/")[5]

cre = Credentials.from_service_account_file(
    "/content/drive/MyDrive/collect_data_yt_toxic_cmt/service_account.json",
    scopes=["https://www.googleapis.com/auth/spreadsheets",
            "https://www.googleapis.com/auth/drive"]
)
gc = gspread.authorize(cre)
sheet = gc.open_by_key(Sheet_id).sheet1
df_link = pd.DataFrame(sheet.get_all_records())


ytb = build("youtube", "v3", developerKey=api_key)

data_raw = "/content/drive/MyDrive/collect_data_yt_toxic_cmt/data_raw"
os.makedirs(data_raw, exist_ok=True)

Function

In [17]:
#video yotube
def get_cmt (vid_id , max_pages = 2):
  # parameters
  # - vid_id : id cua video ytb
  # - max_pages : so luong trang toi da chua cmt / 1 vid
  # Local variable
  # - cmts : list chua cmt
  # - page token : list chua token cua page
  # - req : request
  # - res : response
  # input : vid_id , max_pages
  # output : dataframe chua cmt
  cmts , page_token = [], None

  for _ in range(max_pages):
    req = ytb.commentThreads().list(
        part = "snippet",
        videoId = vid_id,
        maxResults = 100,
        textFormat = "plainText",
        pageToken= page_token
    )

    res = req.execute()
    for item in res.get("item" , []):
      c = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
      cmts.append(c)

    page_token = res.get("nextPageToken")
    if not page_token:
      break

  return pd.DataFrame(cmts)

#replay live chat
def get_livechat(url):
    # Parameters:
    # - url : url cua video
    # Local variable:
    # -ydl_opts : dict
    #     Cấu hình cho yt-dlp, gồm:
    #     - skip_download: True (không tải video, chỉ lấy metadata/subtitles)
    #     - writesubtitles: True (ghi phụ đề / live chat)
    #     - subtitleslangs: ["live_chat"] (chỉ lấy live chat)
    # -ydl : YoutubeDL object
    # json_file : str
    #     Tên file `.live_chat.json` mới nhất được yt-dlp tạo ra.
    # comments : list
    #     List chứa các comment dạng dict {"comment_id": ..., "text": ...}.
    # f : file object
    #     File JSON live chat mở ra để đọc.
    # data : dict
    #     1 dòng dữ liệu JSON từ file.
    # act : dict
    #     1 action trong live chat (thường là addChatItemAction).
    # renderer : dict or None
    #     Renderer của message, chứa id và text comment.
    # cid : str
    #     ID của comment trong live chat.
    # runs : list
    #     Danh sách các đoạn text trong message.
    # text : str
    #     Nội dung comment ghép từ các run.

    # Input:
    # -url
    # Output:
    #     DataFrame chứa các comment live chat

    ydl_opts = {
        "skip_download": True,
        "writesubtitles": True,
        "subtitleslangs": ["live_chat"]
    }

    with YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    json_file = max(glob.glob("*.live_chat.json"), key=os.path.getctime)

    comments = []
    with open(json_file, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            for act in data.get("replayChatItemAction", {}).get("actions", []):
                renderer = act.get("addChatItemAction", {}).get("item", {}).get("liveChatTextMessageRenderer")
                if renderer:
                    cid = renderer.get("id")
                    runs = renderer.get("message", {}).get("runs", [])
                    text = "".join(r.get("text", "") for r in runs)
                    comments.append({"comment_id": cid, "text": text})

    return pd.DataFrame(comments)


Main Pipeline

In [18]:
all_files = []

for i, row in df_link.iterrows():
    stt, link, t, vid = row["STT"], row["Link"], row["Type"], row["ID"]

    if t == 0 and vid:
        out_file = f"{data_raw}/vid_{vid}.csv"
        if not os.path.exists(out_file):
            print(f"[{stt}] Lấy comment video {vid} ...")
            df = get_cmt(vid)
            df.to_csv(out_file, index=False, encoding="utf-8-sig")
        else:
            print(f"[{stt}]  Đã có file {out_file}, bỏ qua")
        all_files.append(out_file)

    elif t == 1 and link:
        out_file = f"{data_raw}/live_{stt}.csv"
        if not os.path.exists(out_file):
            print(f"[{stt}]  Lấy live chat {link} ...")
            df = get_livechat(link)
            df.to_csv(out_file, index=False, encoding="utf-8-sig")
        else:
            print(f"[{stt}]  Đã có file {out_file}, bỏ qua")
        all_files.append(out_file)

print("SUCCESS")

[1]  Đã có file /content/drive/MyDrive/collect_data_yt_toxic_cmt/data_raw/live_1.csv, bỏ qua
[9]  Đã có file /content/drive/MyDrive/collect_data_yt_toxic_cmt/data_raw/live_9.csv, bỏ qua
[2]  Đã có file /content/drive/MyDrive/collect_data_yt_toxic_cmt/data_raw/vid_Z7NHEDDysa0.csv, bỏ qua
[3]  Đã có file /content/drive/MyDrive/collect_data_yt_toxic_cmt/data_raw/vid_3f74WqUicKk.csv, bỏ qua
[4]  Đã có file /content/drive/MyDrive/collect_data_yt_toxic_cmt/data_raw/vid_scR6QC-pFpg.csv, bỏ qua
[5]  Đã có file /content/drive/MyDrive/collect_data_yt_toxic_cmt/data_raw/vid_XrKX2iAIQ14.csv, bỏ qua
[6]  Đã có file /content/drive/MyDrive/collect_data_yt_toxic_cmt/data_raw/vid_p8v2BClNYkc.csv, bỏ qua
[7]  Đã có file /content/drive/MyDrive/collect_data_yt_toxic_cmt/data_raw/vid_U82nFgwCZlI.csv, bỏ qua
[8]  Đã có file /content/drive/MyDrive/collect_data_yt_toxic_cmt/data_raw/vid_bHQLFy-QNRI.csv, bỏ qua
[10]  Đã có file /content/drive/MyDrive/collect_data_yt_toxic_cmt/data_raw/vid_d6_nW3gv59M.csv, bỏ q

Summary

# Báo cáo Thu thập Comment YouTube

**Người thực hiện:** Do Huy  
**Ngày:** 16/09/2025

## 1. Pipeline
1. **Đọc danh sách video/link từ Google Sheet**  
2. **Thu thập dữ liệu**  
   - Comment thường → YouTube API  
   - Live chat → yt-dlp, lưu phụ đề replay  
3. **Lưu dữ liệu thô**  
   - Định dạng: CSV  
   - Thư mục: `data_raw/`  
   - Cột: `comment_id`, `text`  
4. **Kiểm tra & ghi log**  
   - Bỏ qua video đã thu thập  
   - Ghi log tiến trình

## 2. Vấn đề gặp phải
- Giới hạn tốc độ API (giải pháp: retry, tạm dừng)  
- Một số live chat không tải được phụ đề  
- Ký tự đặc biệt/emoji (giải pháp: lưu CSV với `utf-8-sig`)  
- API key được lưu an toàn trong `.env`

## 3. Kế hoạch tiếp theo
- Làm sạch dữ liệu: loại bỏ duplicate, comment trống  
